In [50]:
# import necessary libraries
import torch
import torch.nn as nn

In [51]:
# create convolutional block
class Conv_Block(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride,padding=0):
        super(Conv_Block,self).__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding)
        self.relu = nn.ReLU()

    def forward(self,x):
        return self.relu(self.conv(x))

In [52]:
# It's a combination of filters with various kernel_size to
# get features with different size. It increases accuracy.
class InceptionBlock(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(InceptionBlock,self).__init__()
        self.branch1 = Conv_Block(in_channels,out_channels[0],kernel_size=1,stride=1)
        self.branch2 = nn.Sequential(Conv_Block(in_channels,out_channels[1],kernel_size=1,stride=1),
                                    Conv_Block(out_channels[1],out_channels[2],kernel_size=3,stride=1,padding=1))
        self.branch3 = nn.Sequential(Conv_Block(in_channels,out_channels[3],kernel_size=1,stride=1),
                                    Conv_Block(out_channels[3],out_channels[4],kernel_size=5,stride=1,padding=2))
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size=(3,3),stride=1,padding=1),
                                    Conv_Block(in_channels,out_channels[5],kernel_size=1,stride=1))
    
    def forward(self,x):
        # 1 branch
        x1 = self.branch1(x)
        # 2 branch  
        x2 = self.branch2(x)
        # 3 branch
        x3 = self.branch3(x)
        # 4 branch
        x4 = self.branch4(x)
        # concatanate branches
        x = torch.cat((x1,x2,x3,x4),dim=1)
        return x

In [53]:
# Distinctive features of GoogleNet:
# 1)InceptionBlock
# 2)Use dimension reduction(1x1 conv)
# To see more, read the paper https://arxiv.org/pdf/1409.4842.pdf
class GoogleNet(nn.Module):
    def __init__(self,in_channels,num_classes):
        super(GoogleNet,self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=(3,3),stride=2,padding=1)
        self.conv = nn.Sequential(
                Conv_Block(in_channels,64,kernel_size=7,stride=2,padding=3),
                self.max_pool,
                Conv_Block(64, 64, kernel_size=1, stride=1),
                Conv_Block(64,192,kernel_size=3,stride=1,padding=1),
                self.max_pool
        )
        # [1x1,3x3_red,3x3,5x5_red,5x5,pool proj]
        self.inception_block1 = nn.Sequential(
                InceptionBlock(192,[64,96,128,16,32,32]),
                InceptionBlock(256,[128,128,192,32,96,64])
        )
        self.inception_block2 = nn.Sequential(
                InceptionBlock(480,[192,96,208,16,48,64]),
                InceptionBlock(512,[160,112,224,24,64,64]),
                InceptionBlock(512,[128,128,256,24,64,64]),
                InceptionBlock(512,[112,144,288,32,64,64]),
                InceptionBlock(528,[256,160,320,32,128,128])
        )
        self.inception_block3 = nn.Sequential(
                InceptionBlock(832,[256,160,320,32,128,128]),
                InceptionBlock(832,[384,192,384,48,128,128])
        )
        self.avg_pool = nn.AvgPool2d(kernel_size=(7,7),stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024,num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        x = self.conv(x)
        # block1
        x = self.inception_block1(x)
        x = self.max_pool(x)
        # block2
        x = self.inception_block2(x)
        x = self.max_pool(x)
        # block3
        x = self.inception_block3(x)
        x = self.avg_pool(x)
        x = self.dropout(x)
        # flatten x
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return self.softmax(x)

In [54]:
# create GoogleNet
def googlenet(img_channels=3,num_classes=1000):
    return GoogleNet(img_channels,num_classes)

In [55]:
# test the net architecture
def test():
  net = googlenet()
  x = torch.rand(2,3,224,224)
  y = net(x)
  print(y.shape)

In [56]:
test()

torch.Size([2, 1000])
